In [70]:
# Import our dependencies
import pandas as pd
import requests
from io import StringIO
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import tensorflow as tf

In [109]:
# I had to create a workaround to use my CSV files uploaded to Google Drive
url = 'https://drive.google.com/file/d/1G3kf8VWH8RJopKLrzzmW3lp2oCnuuw-6/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
df = pd.read_csv(path)
# df = pd.read_csv(path, usecols=lambda x: x.startswith('RegionName') or ('/2000' in x) or ('/2023' in x))

print(df.head())

     RegionName      36556      36585      36616      36646      36677  \
0         Tulsa   75173.10   74895.20   75050.84   75344.35   75939.06   
1  Broken Arrow  119608.60  119273.43  119726.92  120396.37  121446.35   
2     Claremore   98698.46   98857.94   99067.24   99449.33   99856.65   
3        Owasso  136822.00  136548.01  136913.26  137566.30  138652.16   
4  Bartlesville   60386.76   60445.58   60548.23   60782.94   61076.19   

       36707      36738      36769      36799  ...      44712      44742  \
0   76411.54   76766.41   77146.07   77512.19  ...  173162.12  175038.87   
1  122121.99  122703.88  123242.53  123725.66  ...  243725.73  246873.22   
2  100320.34  100862.33  101345.51  101741.60  ...  223035.10  225944.42   
3  139449.46  140030.43  140410.21  140775.77  ...  270843.03  274227.06   
4   61356.45   61558.88   61750.92   61967.68  ...  147570.70  149264.45   

       44773      44804      44834      44865      44895      44926  \
0  176847.66  177963.30  17

ValueError: time data '75173' does not match format '%m/%d/%Y' (match)

In [106]:
df.to_csv('../resources/output2.csv', index=False)

In [100]:
df.to_csv('../resources/output.csv', index=False)

In [94]:
X = df.iloc[:, 8:-1]
df_dates = df.loc[:, '1/31/2000':'2/28/2023']


In [96]:
print(df_dates.head())

      1/31/2000     2/29/2000     3/31/2000     4/30/2000     5/31/2000  \
0   $75,173.10    $74,895.20    $75,050.84    $75,344.35    $75,939.06    
1  $119,608.60   $119,273.43   $119,726.92   $120,396.37   $121,446.35    
2   $98,698.46    $98,857.94    $99,067.24    $99,449.33    $99,856.65    
3  $136,822.00   $136,548.01   $136,913.26   $137,566.30   $138,652.16    
4   $60,386.76    $60,445.58    $60,548.23    $60,782.94    $61,076.19    

      6/30/2000     7/31/2000     8/31/2000     9/30/2000    10/31/2000  ...  \
0   $76,411.54    $76,766.41    $77,146.07    $77,512.19    $77,869.52   ...   
1  $122,121.99   $122,703.88   $123,242.53   $123,725.66   $124,102.33   ...   
2  $100,320.34   $100,862.33   $101,345.51   $101,741.60   $102,031.56   ...   
3  $139,449.46   $140,030.43   $140,410.21   $140,775.77   $141,122.60   ...   
4   $61,356.45    $61,558.88    $61,750.92    $61,967.68    $62,189.98   ...   

      5/31/2022     6/30/2022     7/31/2022     8/31/2022     9/30/2

In [72]:
# One-hot encode the RegionName column
X = pd.get_dummies(df['RegionName'])

In [73]:
# Add the date columns to the feature set
X = pd.concat([X, df.iloc[:, 6:]], axis=1)

In [66]:
# Remove any rows with missing values
# X.dropna(inplace=True)

In [78]:
# Split the data into features (X) and target (y)
y = X.mean(axis=1)
X = df.drop(columns=['RegionName', 'Metro', 'RegionType', 'StateName', 'State', 'CountyName'])

C:\Users\steve41320\AppData\Local\Temp\ipykernel_28564\2628955915.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  y = X.mean(axis=1)


In [79]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [80]:
X.isnull().sum()

RegionID      4
SizeRank      4
1/31/2000     4
2/29/2000     4
3/31/2000     4
             ..
10/31/2022    4
11/30/2022    4
12/31/2022    4
1/31/2023     4
2/28/2023     4
Length: 280, dtype: int64

In [81]:
# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

ValueError: could not convert string to float: '$15,968.87 '

In [ ]:
# Fill in missing values
imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)
y_train = imputer.fit_transform(y_train.values.reshape(-1, 1))
y_test = imputer.transform(y_test.values.reshape(-1, 1))

In [ ]:
# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=64, activation='relu', input_shape=[X_train.shape[1]]),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=1)
])

In [ ]:
# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), verbose=1)

In [ ]:
# Evaluate the model
y_pred = model.predict(X_test)
print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_pred)))
print('R-squared:', r2_score(y_test, y_pred))

In [17]:
df['AveragePrice'] = df.iloc[:, 8:].mean(axis=1)

In [18]:
X = df.drop(columns=['RegionName'])
y = df['AveragePrice']

In [19]:
# Exclude non-numerical columns from X
X = df.select_dtypes(include=[np.number]).drop(columns=['RegionID', 'SizeRank'])

In [20]:
# Scale the features using StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [21]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=0)

In [22]:
# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=64, activation='relu', input_shape=[X_train.shape[1]]),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=1)
])

In [23]:
# Compile the model
model.compile(optimizer='adam', loss='mse')

In [25]:
print(np.isnan(X_train).any())
print(np.isnan(y_train).any())

True
False


In [34]:
X_train = X_train.reshape(-1, 1)
y_train = y_train.reshape(-1, 1)


AttributeError: 'Series' object has no attribute 'reshape'

In [29]:
imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
y_train = imputer.fit_transform(y_train)

AttributeError: 'Series' object has no attribute 'reshape'

In [24]:
# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
1/1 [==============================] - 1s 752ms/step - loss: nan - val_loss: nan
Epoch 2/100
1/1 [==============================] - 0s 30ms/step - loss: nan - val_loss: nan
Epoch 3/100
1/1 [==============================] - 0s 32ms/step - loss: nan - val_loss: nan
Epoch 4/100
1/1 [==============================] - 0s 34ms/step - loss: nan - val_loss: nan
Epoch 5/100
1/1 [==============================] - 0s 39ms/step - loss: nan - val_loss: nan
Epoch 6/100
1/1 [==============================] - 0s 48ms/step - loss: nan - val_loss: nan
Epoch 7/100
1/1 [==============================] - 0s 43ms/step - loss: nan - val_loss: nan
Epoch 8/100
1/1 [==============================] - 0s 39ms/step - loss: nan - val_loss: nan
Epoch 9/100
1/1 [==============================] - 0s 47ms/step - loss: nan - val_loss: nan
Epoch 10/100
1/1 [==============================] - 0s 47ms/step - loss: nan - val_loss: nan
Epoch 11/100
1/1 [==============================] - 0s 55ms/step - loss: nan -

1/1 [==============================] - 0s 38ms/step - loss: nan - val_loss: nan
Epoch 90/100
1/1 [==============================] - 0s 39ms/step - loss: nan - val_loss: nan
Epoch 91/100
1/1 [==============================] - 0s 38ms/step - loss: nan - val_loss: nan
Epoch 92/100
1/1 [==============================] - 0s 40ms/step - loss: nan - val_loss: nan
Epoch 93/100
1/1 [==============================] - 0s 35ms/step - loss: nan - val_loss: nan
Epoch 94/100
1/1 [==============================] - 0s 35ms/step - loss: nan - val_loss: nan
Epoch 95/100
1/1 [==============================] - 0s 37ms/step - loss: nan - val_loss: nan
Epoch 96/100
1/1 [==============================] - 0s 35ms/step - loss: nan - val_loss: nan
Epoch 97/100
1/1 [==============================] - 0s 37ms/step - loss: nan - val_loss: nan
Epoch 98/100
1/1 [==============================] - 0s 34ms/step - loss: nan - val_loss: nan
Epoch 99/100
1/1 [==============================] - 0s 33ms/step - loss: nan - val_

In [ ]:
# Evaluate the model on the testing set
y_pred = model.predict(X_test)
print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_pred)))
print('R-squared:', r2_score(y_test, y_pred))